# LightGBM Classifier

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
import pandas as pd
dataset = pd.read_csv('churn_modelling.csv')

In [4]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Checking missing data

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


## Handling categorical variables

CustinerID abd Surname Columns

In [7]:
dataset.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


Geography column

In [11]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [12]:
geography_dummies = pd.get_dummies(dataset['Geography'], drop_first = True)

In [13]:
geography_dummies = geography_dummies.astype(int)

In [14]:
geography_dummies

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,0


In [15]:
dataset = pd.concat([geography_dummies, dataset], axis = 1)

In [16]:
dataset.head()

,Germany,Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
dataset.drop(['Geography'], axis = 1, inplace = True)

In [19]:
dataset.head()

,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,Female,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,Female,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,Female,43,2,125510.82,1,1,1,79084.10,0


Gender column

In [22]:
dataset['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [23]:
dataset['Gender'] = dataset['Gender'].apply(lambda x: 0 if 'Female' else 1)

In [24]:
dataset.head()

,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,0,619,0,42,2,0.00,1,1,1,101348.88,1
1,0,1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,0,0,502,0,42,8,159660.80,3,1,0,113931.57,1
3,0,0,699,0,39,1,0.00,2,0,0,93826.63,0
4,0,1,850,0,43,2,125510.82,1,1,1,79084.10,0


### Creating the Training Set and the Test Set

Getting the inputs and output

In [25]:
X = dataset.iloc[:, :-1].values

In [26]:
y = dataset.iloc[:, -1].values

In [27]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [28]:
y

array([1, 0, 1, ..., 1, 1, 0])

Getting the Training Set and the Test Set

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)